---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 4 - Document Similarity & Topic Modelling

## Part 1 - Document Similarity

For the first part of this assignment, you will complete the functions `doc_to_synsets` and `similarity_score` which will be used by `document_path_similarity` to find the path similarity between two documents.

The following functions are provided:
* **`convert_tag:`** converts the tag given by `nltk.pos_tag` to a tag used by `wordnet.synsets`. You will need to use this function in `doc_to_synsets`.
* **`document_path_similarity:`** computes the symmetrical path similarity between two documents by finding the synsets in each document using `doc_to_synsets`, then computing similarities using `similarity_score`.

You will need to finish writing the following functions:
* **`doc_to_synsets:`** returns a list of synsets in document. This function should first tokenize and part of speech tag the document using `nltk.word_tokenize` and `nltk.pos_tag`. Then it should find each tokens corresponding synset using `wn.synsets(token, wordnet_tag)`. The first synset match should be used. If there is no match, that token is skipped.
* **`similarity_score:`** returns the normalized similarity score of a list of synsets (s1) onto a second list of synsets (s2). For each synset in s1, find the synset in s2 with the largest similarity value. Sum all of the largest similarity values together and normalize this value by dividing it by the number of largest similarity values found. Be careful with data types, which should be floats. Missing values should be ignored.

Once `doc_to_synsets` and `similarity_score` have been completed, submit to the autograder which will run `test_document_path_similarity` to test that these functions are running correctly. 

*Do not modify the functions `convert_tag`, `document_path_similarity`, and `test_document_path_similarity`.*

In [1]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    tokens = nltk.word_tokenize(doc)
    pos   = nltk.pos_tag(tokens)
    
    synsets = []
    for elem in pos:
        # converted_tag(pos_tag)
        wordnet_tag = convert_tag(elem[1])
        # wn.synsets(token, wn_usable_converted_tag)
        synset = wn.synsets(elem[0],wordnet_tag)
        if len(synset) > 0:
            synsets.append(synset[0])
    print('doc_to_synsets for doc <{}> is \n {}\n'.format(doc,synsets))
    return synsets


def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    largest_similarity_values = []
    for syn1 in s1:

        # for each synset in s1 we calc path similarity for
        # ALL element in s2 & append in similarity_values
        
        #similarity_values = [ syn2.path_similarity(syn1) for syn2 in s2 if syn2.path_similarity(syn1) is not None]
        similarity_values = []
        for syn2 in s2:
            if syn1.path_similarity(syn2) != None:
                sim_val = syn1.path_similarity(syn2)
                similarity_values.append(sim_val)
        if len(similarity_values) != 0:
            largest_similarity_values.append( max(similarity_values) )
            
        print('Synset: <{}> in s1\n sim_val: {} \n largest_sim_values: {}\n'.format(syn1,similarity_values,largest_similarity_values))

    return sum(largest_similarity_values) / len(largest_similarity_values)


def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

doc_to_synsets('Fish are nvqjp friends.')
synsets1 = doc_to_synsets('I like cats')
synsets2 = doc_to_synsets('I like dogs')
similarity_score(synsets1, synsets2)

doc_to_synsets for doc <Fish are nvqjp friends.> is 
 [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]

doc_to_synsets for doc <I like cats> is 
 [Synset('iodine.n.01'), Synset('wish.v.02'), Synset('cat.n.01')]

doc_to_synsets for doc <I like dogs> is 
 [Synset('iodine.n.01'), Synset('wish.v.02'), Synset('dog.n.01')]

Synset: <Synset('iodine.n.01')> in s1
 sim_val: [1.0, 0.08333333333333333] 
 largest_sim_values: [1.0]

Synset: <Synset('wish.v.02')> in s1
 sim_val: [0.125, 1.0, 0.08333333333333333] 
 largest_sim_values: [1.0, 1.0]

Synset: <Synset('cat.n.01')> in s1
 sim_val: [0.058823529411764705, 0.2] 
 largest_sim_values: [1.0, 1.0, 0.2]



0.7333333333333334

### test_document_path_similarity

Use this function to check if doc_to_synsets and similarity_score are correct.

*This function should return the similarity score as a float.*

In [2]:
def test_document_path_similarity():
    doc1 = 'This is a function to test document_path_similarity.'
    doc2 = 'Use this function to see if your code in doc_to_synsets \
    and similarity_score is correct!'
    return np.float(document_path_similarity(doc1, doc2))
test_document_path_similarity()

doc_to_synsets for doc <This is a function to test document_path_similarity.> is 
 [Synset('be.v.01'), Synset('angstrom.n.01'), Synset('function.n.01'), Synset('test.v.01')]

doc_to_synsets for doc <Use this function to see if your code in doc_to_synsets     and similarity_score is correct!> is 
 [Synset('use.v.01'), Synset('function.n.01'), Synset('see.v.01'), Synset('code.n.01'), Synset('inch.n.01'), Synset('be.v.01'), Synset('correct.a.01')]

Synset: <Synset('be.v.01')> in s1
 sim_val: [0.3333333333333333, 0.14285714285714285, 0.25, 0.14285714285714285, 0.1111111111111111, 1.0, 0.3333333333333333] 
 largest_sim_values: [1.0]

Synset: <Synset('angstrom.n.01')> in s1
 sim_val: [0.1, 0.1, 0.25] 
 largest_sim_values: [1.0, 0.25]

Synset: <Synset('function.n.01')> in s1
 sim_val: [1.0, 0.14285714285714285, 0.1111111111111111] 
 largest_sim_values: [1.0, 0.25, 1.0]

Synset: <Synset('test.v.01')> in s1
 sim_val: [0.2, 0.1111111111111111, 0.16666666666666666, 0.1111111111111111, 0.090909090

0.554265873015873

<br>
___
`paraphrases` is a DataFrame which contains the following columns: `Quality`, `D1`, and `D2`.

`Quality` is an indicator variable which indicates if the two documents `D1` and `D2` are paraphrases of one another (1 for paraphrase, 0 for not paraphrase).

In [27]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('paraphrases.csv')
paraphrases.head()

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...


___

### most_similar_docs

Using `document_path_similarity`, find the pair of documents in paraphrases which has the maximum similarity score.

*This function should return a tuple `(D1, D2, similarity_score)`*

In [29]:
def most_similar_docs():
    df = paraphrases.copy()
    df['similarity_score'] = df.apply(lambda x: document_path_similarity(x['D2'],x['D1']),axis= 1)
    df = df.iloc[:,1:]
    return tuple(df.iloc[ df['similarity_score'].argmax() ])
most_similar_docs()

doc_to_synsets for doc <Ms Stewart, 61, its chief executive officer and chairwoman, did not attend.
> is 
 [Synset('multiple_sclerosis.n.01'), Synset('stewart.n.01'), Synset('sixty-one.s.01'), Synset('information_technology.n.01'), Synset('chief.s.01'), Synset('executive.a.01'), Synset('military_officer.n.01'), Synset('president.n.04'), Synset('make.v.01'), Synset('not.r.01'), Synset('attend.v.01')]

doc_to_synsets for doc <Ms Stewart, the chief executive, was not expected to attend.> is 
 [Synset('multiple_sclerosis.n.01'), Synset('stewart.n.01'), Synset('chief.s.01'), Synset('executive.n.01'), Synset('be.v.01'), Synset('not.r.01'), Synset('expect.v.01'), Synset('attend.v.01')]

Synset: <Synset('multiple_sclerosis.n.01')> in s1
 sim_val: [1.0, 0.0625, 0.0625] 
 largest_sim_values: [1.0]

Synset: <Synset('stewart.n.01')> in s1
 sim_val: [0.0625, 1.0, 0.1111111111111111] 
 largest_sim_values: [1.0, 1.0]

Synset: <Synset('sixty-one.s.01')> in s1
 sim_val: [] 
 largest_sim_values: [1.0, 1

Synset: <Synset('angstrom.n.01')> in s1
 sim_val: [0.07692307692307693, 0.058823529411764705, 0.058823529411764705, 0.07142857142857142, 0.07692307692307693, 0.1111111111111111, 0.1111111111111111, 1.0, 0.06666666666666667, 0.07692307692307693, 0.07142857142857142] 
 largest_sim_values: [0.09090909090909091, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Synset: <Synset('current.a.01')> in s1
 sim_val: [1.0] 
 largest_sim_values: [0.09090909090909091, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Synset: <Synset('photograph.n.01')> in s1
 sim_val: [0.08333333333333333, 0.07692307692307693, 0.1111111111111111, 0.0625, 0.1, 0.07692307692307693, 0.07692307692307693, 0.06666666666666667, 1.0, 0.06666666666666667, 0.09090909090909091] 
 largest_sim_values: [0.09090909090909091, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Synset: <Synset('address.n.01')> in s1
 sim_val: [0.07692307692307693, 0.058823529411764705, 0.0

Synset: <Synset('browser.n.01')> in s1
 sim_val: [0.07692307692307693, 0.07142857142857142, 0.06666666666666667, 0.07692307692307693, 0.2, 1.0, 0.07142857142857142, 0.1, 0.08333333333333333, 0.08333333333333333, 0.05555555555555555, 0.08333333333333333, 0.08333333333333333] 
 largest_sim_values: [1.0, 0.08333333333333333, 1.0, 1.0, 0.25, 0.1111111111111111, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Synset: <Synset('mention.v.01')> in s1
 sim_val: [0.08333333333333333, 0.16666666666666666, 0.14285714285714285, 0.07692307692307693, 0.07142857142857142, 0.06666666666666667, 0.09090909090909091, 0.07692307692307693, 0.07692307692307693, 0.08333333333333333, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.058823529411764705, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.07142857142857142, 0.09090909090909091] 
 largest_sim_values: [1.0, 0.08333333333333333, 1.0, 1.0, 0.25, 0.1111111111111111, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.16666666666666666]

Synset: <Synset('compe

Synset: <Synset('united_states.n.01')> in s1
 sim_val: [0.05555555555555555, 0.0625, 0.0625, 0.0625, 0.07692307692307693, 0.07692307692307693, 0.0625, 0.125, 0.05263157894736842] 
 largest_sim_values: [0.125]

Synset: <Synset('particular.s.01')> in s1
 sim_val: [1.0] 
 largest_sim_values: [0.125, 1.0]

Synset: <Synset('force.n.01')> in s1
 sim_val: [0.07692307692307693, 1.0, 0.09090909090909091, 0.06666666666666667, 0.1111111111111111, 0.07142857142857142, 0.09090909090909091, 0.09090909090909091, 0.07142857142857142] 
 largest_sim_values: [0.125, 1.0, 1.0]

Synset: <Synset('military_personnel.n.01')> in s1
 sim_val: [0.14285714285714285, 0.09090909090909091, 1.0, 0.06666666666666667, 0.14285714285714285, 0.07142857142857142, 0.09090909090909091, 0.09090909090909091, 0.07142857142857142] 
 largest_sim_values: [0.125, 1.0, 1.0, 1.0]

Synset: <Synset('be.v.01')> in s1
 sim_val: [0.09090909090909091, 0.3333333333333333, 0.1111111111111111, 0.1111111111111111, 1.0, 0.125, 0.333333333333333

('"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.',
 '"Iran should be on notice that attempts to remake Iraq in Iran\'s image will be aggressively put down," he said.\n',
 0.9753086419753086)

### label_accuracy

Provide labels for the twenty pairs of documents by computing the similarity for each pair using `document_path_similarity`. Let the classifier rule be that if the score is greater than 0.75, label is paraphrase (1), else label is not paraphrase (0). Report accuracy of the classifier using scikit-learn's accuracy_score.

*This function should return a float.*

In [13]:
def label_accuracy():
    from sklearn.metrics import accuracy_score
    
    df = paraphrases.copy()
    df['similarity_score'] = df[ ['D1','D2'] ].apply(lambda x: document_path_similarity(x[0],x[1]),axis= 1)
    df['label'] = df.apply(lambda x: 1 if x['similarity_score'] > 0.75 else 0, axis =1)
    
     
    return accuracy_score(df['Quality'], df['label'])
label_accuracy()

doc_to_synsets for doc <"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.> is 
 [Synset('indeed.r.01'), Synset('iran.n.01'), Synset('be.v.01'), Synset('put.v.01'), Synset('on.a.01'), Synset('notice.n.01'), Synset('attempt.n.01'), Synset('try.v.01'), Synset('remake.v.01'), Synset('iraq.n.01'), Synset('inch.n.01'), Synset('image.n.01'), Synset('volition.n.01'), Synset('be.v.01'), Synset('aggressively.r.01'), Synset('put.v.01'), Synset('down.r.01'), Synset('helium.n.01'), Synset('state.v.01')]

doc_to_synsets for doc <"Iran should be on notice that attempts to remake Iraq in Iran's image will be aggressively put down," he said.
> is 
 [Synset('iran.n.01'), Synset('be.v.01'), Synset('on.a.01'), Synset('notice.n.01'), Synset('try.v.01'), Synset('remake.v.01'), Synset('iraq.n.01'), Synset('inch.n.01'), Synset('iran.n.01'), Synset('image.n.01'), Synset('volition.n.01'), Synset('be.v.01'), Synset('aggressively.r.01'

Synset: <Synset('gross.n.02')> in s1
 sim_val: [0.07692307692307693, 1.0, 0.07692307692307693, 0.09090909090909091, 0.0625, 0.07142857142857142, 0.08333333333333333, 0.07692307692307693, 0.09090909090909091, 0.09090909090909091] 
 largest_sim_values: [1.0, 1.0, 1.0, 1.0, 1.0]

Synset: <Synset('recovery.n.01')> in s1
 sim_val: [0.07692307692307693, 0.07692307692307693, 1.0, 0.09090909090909091, 0.05555555555555555, 0.0625, 0.07142857142857142, 0.06666666666666667, 0.07692307692307693, 0.07692307692307693] 
 largest_sim_values: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Synset: <Synset('volition.n.01')> in s1
 sim_val: [0.1111111111111111, 0.09090909090909091, 0.09090909090909091, 1.0, 0.0625, 0.07142857142857142, 0.08333333333333333, 0.07692307692307693, 0.09090909090909091, 0.09090909090909091] 
 largest_sim_values: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Synset: <Synset('be.v.01')> in s1
 sim_val: [0.09090909090909091, 0.5, 1.0, 0.25, 0.1111111111111111, 0.09090909090909091, 0.1111111111111111, 1.0

Synset: <Synset('money.n.01')> in s1
 sim_val: [0.1, 0.07692307692307693, 0.1, 0.07692307692307693, 0.07692307692307693, 0.1, 1.0, 0.08333333333333333] 
 largest_sim_values: [1.0, 1.0, 0.16666666666666666, 0.14285714285714285, 1.0, 0.08333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Synset: <Synset('coerce.v.01')> in s1
 sim_val: [0.2, 0.2, 0.2, 0.2, 0.2, 0.1, 0.08333333333333333, 0.2, 0.08333333333333333, 0.08333333333333333, 0.07692307692307693, 0.2, 0.2, 0.08333333333333333, 0.2, 0.09090909090909091, 0.1111111111111111, 0.08333333333333333] 
 largest_sim_values: [1.0, 1.0, 0.16666666666666666, 0.14285714285714285, 1.0, 0.08333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2]

Synset: <Synset('angstrom.n.01')> in s1
 sim_val: [0.09090909090909091, 0.07142857142857142, 1.0, 0.07142857142857142, 0.07142857142857142, 0.1111111111111111, 0.1, 0.07692307692307693] 
 largest_sim_values: [1.0, 1.0, 0.16666666666666666, 0.14285714285714285, 1.0, 0.08333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 1

Synset: <Synset('symptom.n.01')> in s1
 sim_val: [1.0, 0.07692307692307693, 0.5, 0.5] 
 largest_sim_values: [0.08333333333333333, 0.25, 0.0625, 1.0]

Synset: <Synset('food.n.01')> in s1
 sim_val: [0.09090909090909091, 0.07692307692307693, 0.08333333333333333, 0.08333333333333333] 
 largest_sim_values: [0.08333333333333333, 0.25, 0.0625, 1.0, 0.09090909090909091]

Synset: <Synset('poisoning.n.01')> in s1
 sim_val: [0.09090909090909091, 0.2, 0.08333333333333333, 0.08333333333333333] 
 largest_sim_values: [0.08333333333333333, 0.25, 0.0625, 1.0, 0.09090909090909091, 0.2]

Synset: <Synset('diarrhea.n.01')> in s1
 sim_val: [0.5, 0.07142857142857142, 1.0, 0.3333333333333333] 
 largest_sim_values: [0.08333333333333333, 0.25, 0.0625, 1.0, 0.09090909090909091, 0.2, 1.0]

Synset: <Synset('vomit.v.01')> in s1
 sim_val: [0.09090909090909091, 0.07692307692307693, 0.2, 0.2, 0.08333333333333333, 0.08333333333333333, 1.0, 0.2] 
 largest_sim_values: [0.08333333333333333, 0.25, 0.0625, 1.0, 0.0909090909

doc_to_synsets for doc <Sun was the lone major vendor to see its shipments decline, falling 2.9 percent to 59,692 units.> is 
 [Synset('sun.n.01'), Synset('be.v.01'), Synset('major.a.01'), Synset('seller.n.01'), Synset('see.v.01'), Synset('information_technology.n.01'), Synset('cargo.n.01'), Synset('decline.n.01'), Synset('fall.v.01'), Synset('percentage.n.01'), Synset('unit_of_measurement.n.01')]

doc_to_synsets for doc <IBM (NYSE: IBM)  was the fastest-growing vendor, with sales jumping 37 percent to 220,000 units.
> is 
 [Synset('new_york_stock_exchange.n.01'), Synset('be.v.01'), Synset('seller.n.01'), Synset('gross_sales.n.01'), Synset('jump.v.01'), Synset('thirty-seven.s.01'), Synset('percentage.n.01'), Synset('unit_of_measurement.n.01')]

Synset: <Synset('sun.n.01')> in s1
 sim_val: [0.07692307692307693, 0.08333333333333333, 0.058823529411764705, 0.06666666666666667, 0.08333333333333333] 
 largest_sim_values: [0.08333333333333333]

Synset: <Synset('be.v.01')> in s1
 sim_val: [0.0

0.8

## Part 2 - Topic Modelling

For the second part of this assignment, you will use Gensim's LDA (Latent Dirichlet Allocation) model to model topics in `newsgroup_data`. You will first need to finish the code in the cell below by using gensim.models.ldamodel.LdaModel constructor to estimate LDA model parameters on the corpus, and save to the variable `ldamodel`. Extract 10 topics using `corpus` and `id_map`, and with `passes=25` and `random_state=34`.

In [16]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open('newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [21]:
 # Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

# Your code here:
ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics = 10, id2word = id_map, passes = 25, random_state = 34)

### lda_topics

Using `ldamodel`, find a list of the 10 topics and the most significant 10 words in each topic. This should be structured as a list of 10 tuples where each tuple takes on the form:

`(9, '0.068*"space" + 0.036*"nasa" + 0.021*"science" + 0.020*"edu" + 0.019*"data" + 0.017*"shuttle" + 0.015*"launch" + 0.015*"available" + 0.014*"center" + 0.014*"sci"')`

for example.

*This function should return a list of tuples.*

In [22]:
def lda_topics():
    
    return  ldamodel.print_topics(num_topics = 10, num_words=10)

lda_topics()

[(0, '0.056*"edu" + 0.043*"com" + 0.033*"thanks" + 0.022*"mail" + 0.021*"know" + 0.020*"does" + 0.014*"info" + 0.012*"monitor" + 0.010*"looking" + 0.010*"don"'), (1, '0.024*"ground" + 0.018*"current" + 0.018*"just" + 0.013*"want" + 0.013*"use" + 0.011*"using" + 0.011*"used" + 0.010*"power" + 0.010*"speed" + 0.010*"output"'), (2, '0.061*"drive" + 0.042*"disk" + 0.033*"scsi" + 0.030*"drives" + 0.028*"hard" + 0.028*"controller" + 0.027*"card" + 0.020*"rom" + 0.018*"floppy" + 0.017*"bus"'), (3, '0.023*"time" + 0.015*"atheism" + 0.014*"list" + 0.013*"left" + 0.012*"alt" + 0.012*"faq" + 0.012*"probably" + 0.011*"know" + 0.011*"send" + 0.010*"months"'), (4, '0.025*"car" + 0.016*"just" + 0.014*"don" + 0.014*"bike" + 0.012*"good" + 0.011*"new" + 0.011*"think" + 0.010*"year" + 0.010*"cars" + 0.010*"time"'), (5, '0.030*"game" + 0.027*"team" + 0.023*"year" + 0.017*"games" + 0.016*"play" + 0.012*"season" + 0.012*"players" + 0.012*"win" + 0.011*"hockey" + 0.011*"good"'), (6, '0.017*"information" + 0

### topic_distribution

For the new document `new_doc`, find the topic distribution. Remember to use vect.transform on the the new doc, and Sparse2Corpus to convert the sparse matrix to gensim corpus.

*This function should return a list of tuples, where each tuple is `(#topic, probability)`*

In [24]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [33]:
def topic_distribution():
    new_doc_vect = vect.transform(new_doc)
    corpus = gensim.matutils.Sparse2Corpus(new_doc_vect, documents_columns = False)
    doc_top = ldamodel.get_document_topics(corpus)
    
    elem = []
    for top in doc_top:
        #print(top)
        elem = top
    
    return elem ;
topic_distribution()

[(0, 0.020003106), (1, 0.020003323), (2, 0.02000128), (3, 0.49670705), (4, 0.020004036), (5, 0.020004129), (6, 0.02000297), (7, 0.020002643), (8, 0.020003127), (9, 0.34326833)]


[(0, 0.020003106),
 (1, 0.020003323),
 (2, 0.02000128),
 (3, 0.49670705),
 (4, 0.020004036),
 (5, 0.020004129),
 (6, 0.02000297),
 (7, 0.020002643),
 (8, 0.020003127),
 (9, 0.34326833)]

### topic_names

From the list of the following given topics, assign topic names to the topics you found. If none of these names best matches the topics you found, create a new 1-3 word "title" for the topic.

Topics: Health, Science, Automobiles, Politics, Government, Travel, Computers & IT, Sports, Business, Society & Lifestyle, Religion, Education.

*This function should return a list of 10 strings.*

In [11]:
def topic_names():
    
    
    return ['Religion','Education','Government','Computer & IT', 'Sports','Automobiles','Travel','Health','Politics','Science']